In [ ]:
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()

In [ ]:
prefix = 'headlines-lda-ntm'

input_data = session.upload_data(path='abcnews-date-text.csv.gz', key_prefix=prefix)
print(input_data)

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

role = sagemaker.get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.c5.2xlarge',
                                     instance_count=1)

In [ ]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code='preprocessing-lda-ntm.py',
    
    inputs=[
        ProcessingInput(
                source=input_data,
                destination='/opt/ml/processing/input')
    ],
    
    outputs=[
        ProcessingOutput(
                output_name='train_data',
                source='/opt/ml/processing/train/')
    ],
    
    arguments=[
               '--filename', 'abcnews-date-text.csv.gz'
    ]
)

In [ ]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])